In this notebook, we demonstrate some basic learnings, the most basic being that what is returned from calling an async function is a coroutine:

In [1]:
async def coroutine_1():
    return 1

c = coroutine_1()

type(c)

coroutine

next we demonstrate how to use `async def` and `await` to perform multiple tasks on one thread. 

In the code below, the event loop is managed by the `asyncio` library. When you use `await asyncio.sleep(0)` in the `counter` function, it effectively yields control back to the event loop. The event loop, in turn, is responsible for managing the execution of different tasks and determining which tasks are ready to be resumed.

Here's how it works step by step:

1. The `counter` coroutine is called with `await asyncio.sleep(0)`. This line introduces an intentional pause in the coroutine's execution, allowing other tasks to run.

2. While the `counter` coroutine is paused, the event loop continues to run and can execute other tasks that are ready to run. The `main` function, being an asynchronous function, also allows the event loop to switch between tasks.

3. After a brief pause due to `await asyncio.sleep(0)`, the `counter` coroutine becomes eligible to be resumed by the event loop. The event loop schedules the coroutine to continue its execution.

4. The event loop manages the execution of tasks based on their status and readiness. When a task is ready to run (e.g., after the sleep), the event loop can resume its execution.

So, the event loop itself is responsible for tracking the state of different tasks, handling their pauses, and determining when they are ready to be executed again. The `await` statements and asynchronous constructs provided by `asyncio` facilitate this coordination between different tasks within the event loop.

`await asyncio.sleep(0)`  command for yielding control to the event loop so that other tasks can run

Storing tasks in a list allows a way to await their completion or check their status using `t.done()`

The ` while True` loop continuously checks the status of tasks in the tasks list. It filters out completed tasks and then checks if there are any tasks left. If there are no tasks left (i.e., all tasks are done), it returns from the main function, effectively ending the program.

The crucial part is await tasks[0], which means the program will pause at this point until the first task in the list (tasks[0]) is completed. This ensures that the loop doesn't proceed until at least one task has finished. 

If you remove this line, the loop won't wait for any task to complete, and it will keep checking the task statuses in a tight loop without giving other tasks a chance to execute. This can lead to a busy-waiting scenario, where the program consumes CPU resources without making progress, and the loop never exits.


In [8]:
import asyncio

# Define an asynchronous function 'counter' that takes a 'name' parameter.
async def counter(name: str):
    # Loop from 0 to 9 (inclusive).
    for i in range(0, 5):
        # Print the name and current value of 'i'.
        print(f"{name}: {i!s}")
        # Pause the execution of this coroutine to allow other tasks to run.
        await asyncio.sleep(0)  # This line will pause the current task and allow other tasks to be executed.

# Define the main asynchronous function.
async def main():
    tasks = []  # Create an empty list to store tasks.

    # Create three tasks and append them to the 'tasks' list.
    for n in range(0, 3):
        tasks.append(asyncio.create_task(counter(f"task{n}")))

    while True:
        # Filter out tasks that have completed (done).
        tasks = [t for t in tasks if not t.done()]

        print('len(tasks)', len(tasks))
        # If all tasks are done, return from the 'main' function.
        if len(tasks) == 0:
            return "done"

        # Wait for the completion of the first task in the 'tasks' list.
        await tasks[0] # without this line there is no stop 

# Run the 'main' function using the asyncio event loop.

#1. script version
# asyncio.run(main())

#2. direct version
#d = await main()

# make a coroutine m first, then await it
m = main()
d = await m

print(d, type(d), type(main))

len(tasks) 3
task0: 0
task1: 0
task2: 0
task0: 1
task1: 1
task2: 1
task0: 2
task1: 2
task2: 2
task0: 3
task1: 3
task2: 3
task0: 4
task1: 4
task2: 4
len(tasks) 0
done <class 'str'> <class 'function'>


/var/folders/ys/z0p9dh5j5j9dxn2k2yhpmmfw0000gn/T/ipykernel_63728/3790617059.py:36: RuntimeWarning: coroutine 'main' was never awaited
  m = main()


To run `async def main()` above:

in a python script you would do  `asyncio.run(main())` such as in scripts/monitor.py

```python
import asyncio

asyncio.run(main())
```

jupyter (IPython ≥ 7.0) is already running an event loop and `RuntimeError: asyncio.run() cannot be called from a running event loop`

so in jupyter you just run:

`await main()`

The `asyncio.run(main())` function call is used to run an asyncio program. It was introduced in Python 3.7 as a simple way to run an asyncio event loop until the given coroutine completes. The `main()` function in your code is the entry point for the asyncio program, and `asyncio.run(main())` is essentially saying "run the asyncio event loop until the `main()` coroutine completes."

In a Jupyter Notebook, the event loop is already running in the background, so you can use `await main()` directly without needing to explicitly call `asyncio.run(main())`. The Jupyter environment often has an event loop running by default, making it more convenient to use the `await` syntax directly.

Here's a breakdown:

1. **`asyncio.run(main())`:** This function sets up a new event loop, runs the `main()` coroutine until it completes, and then closes the event loop. It's a simple and convenient way to run asyncio code.

2. **`await main()`:** In a Jupyter Notebook or any environment where an event loop is already running, you can use the `await` syntax directly. This assumes that the event loop is already set up and running in the background. The `await main()` line means "wait for the `main()` coroutine to complete before proceeding." The event loop in the background manages the execution of the coroutine.

In the context of a Jupyter Notebook, when you use `await main()`, you are essentially running an asynchronous function (`main()` in this case) in an environment where an event loop is already running.

Here's what happens step by step:

1. **Jupyter Notebooks and Event Loop:** Jupyter Notebooks often run an event loop in the background. This event loop allows you to run asynchronous code using the `await` syntax directly in a notebook cell.

2. **`main()` as an Asynchronous Function:** The `main()` function is an asynchronous function (coroutine). It contains asynchronous operations like `await asyncio.sleep(0)` and `await tasks[0]`. These operations would typically be used in an environment with an event loop.

3. **`await main()`:** By using `await main()` in a Jupyter Notebook cell, you are telling the event loop to execute the `main()` coroutine. The `await` keyword essentially says, "pause here and allow other tasks to run until `main()` completes."

4. **Event Loop Execution:** The event loop in the Jupyter environment manages the execution of asynchronous tasks. When you `await main()`, it lets the event loop run other tasks while waiting for the completion of the `main()` coroutine.

In summary, `await main()` is a way to run asynchronous code in a Jupyter Notebook where an event loop is already running. It allows you to interact with and await the completion of asynchronous tasks in a notebook cell.

In summary, `asyncio.run(main())` is a way to run asyncio code in a script or a non-asyncio context, while `await main()` is a way to run asyncio code in an environment where an event loop is already running. In a Jupyter Notebook, the event loop is often running, so you can use `await` directly.